# Dataset Tool

## Setup Faster RCNN Model

In [1]:
import torchvision
import torch

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# For training
images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)

print(images[0].size())
# For inference
model.eval()
predictions = model([images[0]])

class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

torch.Size([3, 600, 1200])


c:\users\sebri\anaconda3\envs\dataset\lib\site-packages\torchvision\ops\boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


In [ ]:
import os
import numpy as np
from PIL import Image, ImageDraw
from torchvision.transforms import ToTensor
from tqdm import tqdm
import pathlib
import pickle


image_dir = './corgis'
pred_dir = os.path.join(image_dir, 'predictions')
pathlib.Path(pred_dir).mkdir(parents=True, exist_ok=True)

batch_size = 10
image_names = os.listdir(image_dir)
iters = len(image_names) // batch_size + 1

for image_name in tqdm(os.listdir(image_dir)):
    image = Image.open(os.path.join(image_dir, image_name))
    image_tensor = ToTensor()(image).unsqueeze(0)
    model.eval()
    predictions = model(image_tensor)
    pickle.dump(predictions[0], open(os.path.join(pred_dir, image_name.split('.')[0] + '.pkl'), "wb" ))

 32%|███████████████████████▋                                                  | 1976/6164 [1:50:57<3:39:04,  3.14s/it]

In [23]:
def remove_videos(image_dir):
    file_names = os.listdir(image_dir)
    for file_name in file_names:
        if os.path.isdir(os.path.join(image_dir, file_name)):
            continue
        ext = file_name.split('.')
        if len(ext) < 2 or ext[1] not in ['jpg', 'jpeg', 'png']:
            os.remove(os.path.join(image_dir, file_name))
            
remove_videos(image_dir)

In [3]:
def create_size_dict(image_dir):
    image_names = os.listdir(image_dir)
    size_dict = {}
    for image_name in tqdm(image_names):
        if os.path.isdir(os.path.join(image_dir, image_name)):
            print('asd')
            continue
        
        image = Image.open(os.path.join(image_dir, image_name))
        w,h = image.size
        size_str = f'{w}x{h}'
        if size_str not in size_dict.keys():
            size_dict[size_str] = [image_name]
        else:
            size_dict[size_str] = size_dict[size_str] + [image_name]
            
    return size_dict

size_dict = create_size_dict(image_dir)

 97%|███████████████████████████████████████████████████████████████████████▌  | 13212/13657 [00:03<00:00, 3754.94it/s]

asd


100%|██████████████████████████████████████████████████████████████████████████| 13657/13657 [00:03<00:00, 3755.39it/s]


In [4]:
removed = 0
remove_keys = []
for size in size_dict.keys():
    w, h = [int(x) for x in size.split('x')]
    if w < 700 and h < 700:
        for image_name in size_dict[size]:
            os.remove(os.path.join(image_dir, image_name))
            removed += 1
        remove_keys.append(size)
        
for key in remove_keys:
    size_dict.pop(key, None)

print(removed)

6134


In [7]:
import hashlib

removed = 0
for size in size_dict.keys():
    if len(size_dict[size]) < 2:
        continue
    
    hashes = []
    image_files = []
    for image_file in size_dict[size]:
        hash_md5 = hashlib.md5()
        with open(os.path.join(image_dir, image_file), "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        hashes.append(hash_md5.hexdigest())
        image_files.append(image_file)
        
    unique_hashes = set(hashes)
    for h in unique_hashes:
        cnt = hashes.count(h)
        if cnt > 1:
            idxs = [i for i in range(len(hashes)) if hashes[i] == h]
            idxs.pop(0)
            for idx in idxs:
                os.remove(os.path.join(image_dir, image_files[idx]))
                removed +=1

print(removed)

1359


In [ ]:
possible_dupes = [size for size in images if len(images[size]) > 1]
for size in possible_dupes:
    hashes = defaultdict(list)
    for fname in images[size]:
        m = md5.new()
        hashes[ m.update( file(fname,'rb').read(10000) ).digest() ] = fname
    for k in hashes:
       if len(hashes[k]) <= 1: continue
       for fname in hashes[k][1:]:
           os.remove(fname)